## neural translate 번역

- 문장 > 단어(토큰)들     : 토큰나이징 tokenizing
- 단어 > 벡터(1차원 배열) : word embedding

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
x = [[0,3,1],[0,5,1],[0,6,1],[7,3,2],[7,4,2]] # 문장 > 심볼
y = [[0,1,2,4],[0,1,2,6],[0,1,2,7],[8,9,4,3],[8,9,5,3]]
len(x), len(y)

(5, 5)

문장 > 토큰나이징 > 토큰(단어)들
나는 남자 다 i am a man
나는 소년 다 i am a boy
나는 소녀 다 i am a girl
너는 남자 ?  are you man ?
너는 여자 ?  are you woman ?

나는 : 0 
다 : 1
? : 2
남자 : 3
여자 : 4
소년 : 5
소녀 : 6
너는 : 7

In [ ]:
i : 0 
am : 1
a : 2
? : 3
man : 4
woman : 5
boy : 6
girl : 7
are : 8
you : 9

In [30]:
x = np.array(x)
y = np.array(y)
x.shape, y.shape, x.dtype, y.dtype

((5, 3), (5, 4), dtype('int32'), dtype('int32'))

In [246]:
embedding_size = 5 # 한 단어는 사이즈 5의 벡터로 변환하겠다
vocabulary_size = np.max(x) + 1 # 한글 전체 단어 수 8
vocabulary_size_target = np.max(y) + 1 # 영어 전체 단어수 10 
vocabulary_size, vocabulary_size_target, vocabulary_size_target.dtype

(8, 10, dtype('int32'))

In [263]:
tf.reset_default_graph()
import tensorflow.contrib.slim as slim

word_embeddings = tf.get_variable('embed',shape=[vocabulary_size, embedding_size])
embedded = tf.nn.embedding_lookup(word_embeddings, x)#(5,3) > (5,3,embedding_size) 
cell_encoder = tf.nn.rnn_cell.GRUCell(100, name='enc')
tf.nn.rnn_cell.DropoutWrapper(cell_encoder, )
outputs, last_state = tf.nn.dynamic_rnn(cell_encoder, embedded, dtype=tf.float32)
cell_decoder = tf.nn.rnn_cell.GRUCell(100, name='dec')
#(5:다섯개의 문장, 4:문장의 단어 수, 단어를 ?개의 배열로 변환해라)
zero_matrix = tf.zeros([5, 4, embedding_size]) 
outputs, last_state = tf.nn.dynamic_rnn(
    cell_decoder, zero_matrix, initial_state=last_state)
out = slim.fully_connected(outputs, 10) # 영어 단어 총 10개
cost = tf.compat.v1.nn.sparse_softmax_cross_entropy_with_logits(logits=out,labels=y)
cost = tf.reduce_mean(cost)
train = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [267]:
for i in range(1000):
    _, c = sess.run([train, cost])
print(i, c)

999 0.69221497


In [271]:
np.set_printoptions(precision=3, suppress=True)
y_cls = np.argmax(sess.run(out), -1)
print(y_cls)

[[0 1 2 0]
 [0 1 2 0]
 [0 1 2 7]
 [8 0 0 3]
 [8 0 0 3]]
